<a href="https://colab.research.google.com/github/xuanhuyid/BigData/blob/DEEEP-LEARNING/7_label_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**7_label_VGG19**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Lib

In [ ]:
import os
import cv2
import numpy as np
import PIL
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import keras
from tensorflow.keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
import tensorflow as tf
import os 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix,f1_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

## Data

### Create data path


In [ ]:

# Gán đường dẫn dữ liệu chính
path_data = '/content/data'

# Gán đường dẫn dữ liệu nén zip
path_data_zip = path_data + '/zip'
path_data_zip_train = path_data_zip + '/train.zip'
path_data_zip_test = path_data_zip + '/test.zip'

# Gán đường dẫn dữ liệu train, validation, test
path_data_train = path_data + '/train'
path_data_test = path_data + '/test'

# Kiểm tra đường dẫn dữ liệu chính có hay không và tạo mới nếu không có
if not os.path.isdir(path_data):
  os.makedirs(path_data)

### Download and unzip data

In [ ]:
# Kiểm tra nếu chưa có đường dẫn chứa data zip thì tải về
# tao bo data train
# test https://drive.google.com/file/d/1kyVl8jaAhiUZWKJE49X7kwC6N28nD14u/view?usp=share_link
# train https://drive.google.com/file/d/1bEmVRd2B0TiKa1JvEpjOlt8fbxpo5GP9/view?usp=share_link
if not os.path.isdir(path_data_zip):
  os.makedirs(path_data_zip)
!gdown --o $path_data_zip_train --id "1bEmVRd2B0TiKa1JvEpjOlt8fbxpo5GP9"
!gdown --o $path_data_zip_test --id "1kyVl8jaAhiUZWKJE49X7kwC6N28nD14u"
if not os.path.isdir(path_data_train):
  !unzip $path_data_zip_train -d $path_data
if not os.path.isdir(path_data_zip_test):
  !unzip $path_data_zip_test -d $path_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/train/CHUA DU DO NGAU/CD (728).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (729).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (730).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (734).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (735).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (736).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (738).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (739).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (740).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (741).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (742).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (743).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (747).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (748).jpg  
  inflating: /content/data/train/CHUA DU DO NGAU/CD (

### Data augmentation

In [ ]:
train_data_gen = ImageDataGenerator(rescale = 1./255.)
test_data_gen = ImageDataGenerator( rescale = 1.0/255. )
image_shape = (224, 224)
batch_size=32

In [ ]:
train_data=train_data_gen.flow_from_directory(path_data_train,target_size=image_shape,color_mode='rgb',batch_size=batch_size,shuffle=False)
test_data=test_data_gen.flow_from_directory(path_data_test,target_size=image_shape,color_mode='rgb',batch_size=batch_size,shuffle=False)

Found 8978 images belonging to 7 classes.
Found 949 images belonging to 7 classes.


In [ ]:
len(train_data.class_indices)

7

### Split y_test

In [ ]:
# arr_0=np.zeros(len(os.listdir(path_data_test+'/fire'),)).reshape(-1,1)
# print(arr_0.shape)
# arr_1=np.ones(len(os.listdir(path_data_test+'/nonfire'),)).reshape(-1,1)
# print(arr_1.shape)
# y_test=np.vstack((arr_0,arr_1))
# print(y_test.shape)
# #print(y_test)

# Model

## VGG19


In [ ]:
base_model = VGG19(input_shape = (224, 224, 3), 
include_top = False,
weights = 'imagenet')
print(base_model.summary())
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

x= layers.Dense(256, activation='relu')(x)

x= layers.Dropout(0.5)(x) 


# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(len(train_data.class_indices), activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

print(model.summary())

model.compile(optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5), loss = 'categorical_crossentropy',metrics = ['acc'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Train model

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/best_model.h5',
                                                      monitor='val_acc',
                                                      save_best_only=True)
vgg19_model = model.fit(train_data, validation_data = test_data, steps_per_epoch = 50, epochs = 20,callbacks=[checkpoint_callback])

Epoch 1/20
50/50 [==============================] - 94s 2s/step - loss: 2.3724 - acc: 0.1513 - val_loss: 1.9201 - val_acc: 0.1907
Epoch 2/20
50/50 [==============================] - 81s 2s/step - loss: 2.0271 - acc: 0.1925 - val_loss: 1.8094 - val_acc: 0.2835
Epoch 3/20
50/50 [==============================] - 75s 2s/step - loss: 1.8196 - acc: 0.3063 - val_loss: 1.7212 - val_acc: 0.3530
Epoch 4/20
50/50 [==============================] - 88s 2s/step - loss: 1.8552 - acc: 0.2900 - val_loss: 1.6780 - val_acc: 0.3688
Epoch 5/20
50/50 [==============================] - 76s 2s/step - loss: 1.7199 - acc: 0.3720 - val_loss: 1.6099 - val_acc: 0.3878
Epoch 6/20
50/50 [==============================] - 79s 2s/step - loss: 1.6968 - acc: 0.3506 - val_loss: 1.6694 - val_acc: 0.3530
Epoch 7/20
50/50 [==============================] - 69s 1s/step - loss: 1.5559 - acc: 0.4531 - val_loss: 1.6903 - val_acc: 0.3762
Epoch 8/20
50/50 [==============================] - 76s 2s/step - loss: 1.6932 - acc: 0.40

In [ ]:
bath  = len(test_data)
pre = []
true = []
for i in range(bath):
  y_pred = model.predict(test_data[i][0])
  for j in y_pred:
    pre.append(np.argmax(j,axis = 0))
  for j in test_data[i][1]:
    true.append(np.argmax(j,axis = 0))
from sklearn.metrics import accuracy_score,f1_score
print(accuracy_score(true,pre),f1_score(true,pre,average = 'macro'),f1_score(true,pre,average = 'weighted'))

1/1 [==============================] - 0s 177ms/step
0.5331928345626976 0.4665692697761539 0.4947147334300292


In [ ]:
model.load_weights('/content/drive/MyDrive/best_model.h5')
bath  = len(test_data)
pre = []
true = []
for i in range(bath):
  y_pred = model.predict(test_data[i][0])
  for j in y_pred:
    pre.append(np.argmax(j,axis = 0))
  for j in test_data[i][1]:
    true.append(np.argmax(j,axis = 0))
from sklearn.metrics import accuracy_score,f1_score
print(accuracy_score(true,pre),f1_score(true,pre,average = 'macro'),f1_score(true,pre,average = 'weighted'))

1/1 [==============================] - 0s 25ms/step
0.5374077976817703 0.43321742951646425 0.4983899241396228
